# Problem 1

### Import Data: CMT Rates

In [1]:
import numpy as np
import pandas as pd
import quandl

In [2]:
token = 'PxHgKF5Bzq8ypAUmtKsg'
CMTs = quandl.get("USTREASURY/YIELD", authtoken=token, start_date="2013-01-01", end_date="2017-01-31")

In [3]:
CMTs.head()

,1 MO,2 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
Date,,,,,,,,,,,,
2013-01-02,0.07,None,0.08,0.12,0.15,0.27,0.37,0.76,1.25,1.86,2.63,3.04
2013-01-03,0.06,None,0.08,0.12,0.15,0.27,0.40,0.81,1.31,1.92,2.70,3.12
2013-01-04,0.06,None,0.07,0.11,0.15,0.27,0.41,0.82,1.32,1.93,2.70,3.10
2013-01-07,0.05,None,0.07,0.11,0.15,0.27,0.41,0.82,1.31,1.92,2.70,3.10
2013-01-08,0.06,None,0.07,0.11,0.14,0.25,0.38,0.79,1.28,1.89,2.66,3.06


In [4]:
global CMTs

In [5]:
# Remove '2 MO'
CMTs = CMTs.drop(columns = '2 MO')

## Step 1
* As shown in Diebold-Li paper, we assume that $\lambda = 0.0609 $.

In [6]:
lamb0 = 0.0609 # The lambda assumed
m = [1,3,6,12,12*2,12*3,12*5,12*7,12*10,12*20,12*30]

In [7]:
import math
from statsmodels.regression.linear_model import OLS

In [8]:
exog0 = np.array([[1,(1-math.exp(-lamb0*i))/(lamb0*i),(1-math.exp(-lamb0*i))/(lamb0*i)-math.exp(-lamb0*i)] for i in m])

In [9]:
demo = pd.DataFrame(columns = ['b1','b2','b3','ssr'])
for j in CMTs.index:
    model = OLS(CMTs.loc[j,:], exog0).fit()
    demo.loc[j,:] = np.append(np.array(model.params),model.ssr)

* **Solution to Step 1**:The demo table stores the **fitted $\beta$s** on specific dates and also records the **ssr(Sum Squared of Residuals)**.

In [10]:
demo.head()

,b1,b2,b3,ssr
2013-01-02,3.10231,-2.75766,-5.45356,0.455388
2013-01-03,3.18981,-2.8519,-5.53614,0.442416
2013-01-04,3.18349,-2.85776,-5.47936,0.415237
2013-01-07,3.1782,-2.85683,-5.46594,0.427124
2013-01-08,3.13969,-2.8088,-5.48885,0.420724


## Step 2

SSR Function:
* exog: $X$ matrix in linear regression
* b: fitted betas, the formula is $\beta = (X^{T}X)^{-1}X^{T}y$
* e: errors, the formula is $error = y-\tilde{y} = y - X\beta$

In [11]:
def SSR(lam):
    ssr = 0
    m = [1,3,6,12,12*2,12*3,12*5,12*7,12*10,12*20,12*30]
    exog = np.array([[1,(1-math.exp(-lam*i))/(lam*i),(1-math.exp(-lam*i))/(lam*i)-math.exp(-lam*i)] for i in m], dtype='float')
    for j in CMTs.index:
        b = np.dot(np.linalg.inv(exog.T.dot(exog)), exog.T.dot(CMTs.loc[j,:]))
        e = CMTs.loc[j,:]-exog.dot(b)
        ssr = ssr + e.T.dot(e)
    return ssr

In [12]:
import scipy as sp

In [13]:
optimize = sp.optimize.minimize(SSR,lamb0)

In [14]:
optimize

      fun: 57.33477907612194
 hess_inv: array([[2.75080097e-06]])
      jac: array([9.53674316e-07])
  message: 'Optimization terminated successfully.'
     nfev: 33
      nit: 7
     njev: 11
   status: 0
  success: True
        x: array([0.0431258])

* **Solution to Step 2**: It shows that the **optimal $\lambda = 0.0431258$**. In the meanwhile, it's total SSR is about 57.33.

In [15]:
lamb = optimize['x'][0]

In [16]:
exog = np.array([[1,(1-math.exp(-lamb*i))/(lamb*i),(1-math.exp(-lamb*i))/(lamb*i)-math.exp(-lamb*i)] for i in m])

In [17]:
factors = pd.DataFrame(columns = ['b1','b2','b3','ssr'], dtype = 'float')
for j in CMTs.index:
    model = OLS(CMTs.loc[j,:], exog).fit()
    factors.loc[j,:] = np.append(np.array(model.params),model.ssr)

* factors table stores the best fitted dynamic $\beta$s and corresponding ssr.

In [18]:
factors.head()

,b1,b2,b3,ssr
2013-01-02,3.423031,-3.200014,-4.891039,0.115209
2013-01-03,3.512073,-3.299002,-4.933246,0.105931
2013-01-04,3.500129,-3.298936,-4.860845,0.095373
2013-01-07,3.495753,-3.297564,-4.855132,0.100726
2013-01-08,3.457841,-3.251722,-4.890358,0.096970


# Problem 2

## Step 1

In [19]:
factors['RMSE'] = np.sqrt(factors['ssr']/len(m))

In [20]:
factors['RMSE'].nlargest(20)

2013-04-04    0.110085
2013-05-02    0.108868
2013-04-03    0.108627
2013-05-01    0.108623
2013-03-27    0.108620
2013-03-28    0.108543
2013-04-26    0.108489
2013-04-30    0.108409
2013-04-29    0.108024
2013-04-05    0.107589
2013-04-22    0.106212
2013-04-09    0.105977
2013-04-17    0.105566
2013-03-01    0.105428
2013-04-24    0.104982
2013-04-18    0.104867
2013-02-28    0.104844
2013-04-15    0.104702
2013-04-23    0.104070
2013-05-16    0.103936
Name: RMSE, dtype: float64

* **Solution to Step 1**: Series above show the largest RMSE and corresponding 20 days and days stores all 20 dates with largest RMSE.

In [21]:
days = factors['RMSE'].nlargest(20).index

## Step 2

According to Diebold-Li's paper, because of two disadvantages of VAR(1), simple **AR(1)** model will be chosen for estimation and forecasts instead of VAR(1).

### Fit AR(1) Model
* AR(1):
$$\beta_{i}(t) = intercept + slope \times \beta_{i}(t-1) + \epsilon_{i}(t) ,i=1,2,3$$
* Forecast using AR(1):
$$\beta_{i}(t) = intercept + slope \times \beta_{i}(t-1) ,i=1,2,3$$

In [22]:
from dateutil.relativedelta import relativedelta

In [23]:
import statsmodels.api as sm
def AR(s):
    """Calculate the AR(1) parameters and Half Life of mean reversion series.
    Methodology: https://quant.stackexchange.com/questions/25086/calculating-half-life-of-mean-reverting-series-with-python
    
    Args:
    s: pandas.Series
    
    Returns:
    Intercept: float
    Slope: float
    HF: float
    """
    x = s.shift()[1:] # Y(t-1)
    y = s[1:] # Y(t)
    x = sm.add_constant(x) #adds intercept terms to X variable for regression
    x.columns = ['const','x'] # Y(t) = intercept + slope *Y(t-1) + e
    model = sm.OLS(y,x).fit()
    
    intercept = model.params[0]
    slope = sm.OLS(y,x).fit().params[1] 
    HF = -math.log(2) / (slope-1) # Half-life = - log(2)/slope
    return intercept, slope, HF

In [47]:
AR_data = pd.DataFrame(columns = np.arange(0,9))
for t in days:
    AR_data.loc[t,np.arange(3)] = AR(factors.loc[t-relativedelta(months = 6):t,'b1'])
    AR_data.loc[t,np.arange(3,6)] = AR(factors.loc[t-relativedelta(months = 6):t,'b2'])
    AR_data.loc[t,np.arange(6,9)] = AR(factors.loc[t-relativedelta(months = 6):t,'b3'])
    
col = []
for i in factors.columns[0:3]:
    for j in ['intercept','slope','HF']:
        col.append(i+'_'+j)
AR_data.columns = col

* AR_data stores AR(1) parameters of all $\beta$s on the 20 dates.

In [25]:
AR_data

,b1_intercept,b1_slope,b1_HF,b2_intercept,b2_slope,b2_HF,b3_intercept,b3_slope,b3_HF
2013-04-04,0.368666,0.896024,6.66639,-0.408237,0.877282,5.6483,-0.414536,0.917937,8.44656
2013-05-02,0.0726951,0.978275,31.9052,-0.111103,0.965231,19.9355,-0.257953,0.94758,13.2231
2013-04-03,0.434377,0.877897,5.67673,-0.478384,0.856701,4.83706,-0.411582,0.918576,8.51285
2013-05-01,0.0803295,0.976127,29.0344,-0.115176,0.96401,19.2595,-0.282674,0.942709,12.0988
2013-03-27,0.463961,0.869968,5.3306,-0.497596,0.851316,4.66188,-0.383588,0.924302,9.15679
2013-03-28,0.46094,0.870787,5.36437,-0.503638,0.849564,4.60758,-0.408222,0.919226,8.58132
2013-04-26,0.111502,0.967394,21.258,-0.148755,0.954001,15.0688,-0.289675,0.941348,11.818
2013-04-30,0.114161,0.966645,20.7812,-0.150751,0.953401,14.8749,-0.293758,0.940527,11.6549
2013-04-29,0.119269,0.965214,19.9261,-0.157014,0.951534,14.3017,-0.310737,0.937194,11.0363
2013-04-05,0.176168,0.949549,13.739,-0.239677,0.927215,9.52321,-0.441922,0.911545,7.83615


### Forecast $\beta$

In [26]:
def forecast_AR(date,data_set,AR):
    forecast = pd.DataFrame(columns = ['b'+str(i) for i in range(1,4)])
    for t in data_set.loc[i+relativedelta(days=1):i+relativedelta(months=1,days=1),:].index:
        for j in forecast.columns:
            forecast.loc[t,j] = AR.loc[i,j+'_intercept']+AR.loc[i,j+'_slope']*data_set.shift().loc[t,j]
    return forecast

In [27]:
forecast_b = {}
for i in days:
    forecast_b[i] = forecast_AR(i, factors, AR_data)

* **Solution to Step 2**: forecast_b stores t+1m forecasted $\beta$s on the 20 dates.

In [28]:
forecast_b[days[0]] # Demo: Forecast betas at 2013-04-04

,b1,b2,b3
2013-04-05,3.39508,-3.169,-5.04231
2013-04-08,3.26795,-3.06174,-4.76566
2013-04-09,3.31331,-3.10883,-4.79693
2013-04-10,3.34841,-3.13908,-4.87543
2013-04-11,3.42427,-3.21106,-4.95383
2013-04-12,3.41553,-3.20444,-4.95158
2013-04-15,3.32324,-3.12244,-4.85964
2013-04-16,3.27714,-3.08053,-4.78651
2013-04-17,3.30909,-3.10587,-4.8006
2013-04-18,3.27959,-3.0828,-4.72829


### Forecast yields

In [29]:
def DNS2Y(beta_set, lamb, months):
    """
    Convert DNS model betas to CMT yields.
    """
    forecast = pd.DataFrame(columns = months.keys())
    for t in beta_set.index:
        for m,n in months.items():
            forecast.loc[t,m] = beta_set.loc[t,'b1'] + beta_set.loc[t,'b2']*(1-math.exp(-lamb*n))/(lamb*n) + beta_set.loc[t,'b3']*((1-math.exp(-lamb*n))/(lamb*n)-math.exp(-lamb*n))
    return forecast

In [30]:
month = dict(zip(CMTs.columns,m))
forecast_y = {}
for i in days:
    forecast_y[i] = DNS2Y(forecast_b[i], lamb, month)

* **Soluntion to Step 2**: forecast_y stores t+1m forecasted CMT yields on the 20 dates.

In [31]:
forecast_y[days[0]].head(5) # Forecast t+5d CMTs Rates from 2013-04-04 

,1 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
2013-04-05,0.187797,0.123157,0.0528633,-0.00989044,0.0708576,0.293356,0.839548,1.32363,1.84588,2.60192,2.86619
2013-04-08,0.171432,0.113052,0.0504056,-0.00221493,0.0846067,0.302572,0.828812,1.29225,1.79094,2.51187,2.76378
2013-04-09,0.170058,0.112394,0.0508744,0.000620796,0.09227,0.314777,0.848528,1.3174,1.82143,2.54967,2.8041
2013-04-10,0.173907,0.114459,0.0507585,-0.00238953,0.0875064,0.311277,0.850662,1.32537,1.83608,2.57426,2.83219
2013-04-11,0.177673,0.118145,0.0546438,0.00280033,0.0975321,0.327379,0.87867,1.36292,1.88348,2.63559,2.89836


In [32]:
forecast_y[days[0]].head(10) # Forecast t+10d CMTs Rates from 2013-04-04 

,1 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
2013-04-05,0.187797,0.123157,0.0528633,-0.00989044,0.0708576,0.293356,0.839548,1.32363,1.84588,2.60192,2.86619
2013-04-08,0.171432,0.113052,0.0504056,-0.00221493,0.0846067,0.302572,0.828812,1.29225,1.79094,2.51187,2.76378
2013-04-09,0.170058,0.112394,0.0508744,0.000620796,0.09227,0.314777,0.848528,1.3174,1.82143,2.54967,2.8041
2013-04-10,0.173907,0.114459,0.0507585,-0.00238953,0.0875064,0.311277,0.850662,1.32537,1.83608,2.57426,2.83219
2013-04-11,0.177673,0.118145,0.0546438,0.00280033,0.0975321,0.327379,0.87867,1.36292,1.88348,2.63559,2.89836
2013-04-12,0.175451,0.11574,0.0519743,-0.00036311,0.093513,0.322662,0.872927,1.3565,1.87643,2.62771,2.89019
2013-04-15,0.16535,0.105831,0.0419682,-0.0116578,0.0769266,0.299228,0.835893,1.30849,1.81705,2.55222,2.80911
2013-04-16,0.161807,0.103391,0.0407776,-0.0115321,0.0765189,0.296059,0.825411,1.29135,1.79264,2.51723,2.77042
2013-04-17,0.168644,0.110719,0.0488484,-0.00198007,0.0888974,0.31096,0.844318,1.31307,1.81708,2.54537,2.79982
2013-04-18,0.163242,0.107155,0.0475657,-0.000115402,0.0931324,0.314582,0.843471,1.30727,1.8055,2.52508,2.77647


In [33]:
forecast_y[days[0]] # Forecast t+1m CMTs Rates from 2013-04-04 

,1 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
2013-04-05,0.187797,0.123157,0.0528633,-0.00989044,0.0708576,0.293356,0.839548,1.32363,1.84588,2.60192,2.86619
2013-04-08,0.171432,0.113052,0.0504056,-0.00221493,0.0846067,0.302572,0.828812,1.29225,1.79094,2.51187,2.76378
2013-04-09,0.170058,0.112394,0.0508744,0.000620796,0.09227,0.314777,0.848528,1.3174,1.82143,2.54967,2.8041
2013-04-10,0.173907,0.114459,0.0507585,-0.00238953,0.0875064,0.311277,0.850662,1.32537,1.83608,2.57426,2.83219
2013-04-11,0.177673,0.118145,0.0546438,0.00280033,0.0975321,0.327379,0.87867,1.36292,1.88348,2.63559,2.89836
2013-04-12,0.175451,0.11574,0.0519743,-0.00036311,0.093513,0.322662,0.872927,1.3565,1.87643,2.62771,2.89019
2013-04-15,0.16535,0.105831,0.0419682,-0.0116578,0.0769266,0.299228,0.835893,1.30849,1.81705,2.55222,2.80911
2013-04-16,0.161807,0.103391,0.0407776,-0.0115321,0.0765189,0.296059,0.825411,1.29135,1.79264,2.51723,2.77042
2013-04-17,0.168644,0.110719,0.0488484,-0.00198007,0.0888974,0.31096,0.844318,1.31307,1.81708,2.54537,2.79982
2013-04-18,0.163242,0.107155,0.0475657,-0.000115402,0.0931324,0.314582,0.843471,1.30727,1.8055,2.52508,2.77647


## Step 3

### Forecast $\beta$ with Random Walk
* Random Walk:
$$\beta_{i}(t) = \beta_{i}(t-1) + \epsilon_{i}(t) ,i=1,2,3$$
* Forecast using AR(1):
$$\beta_{i}(t) = \beta_{i}(t-1) ,i=1,2,3$$

In [34]:
def forecast_RW(date,date_set):
    forecast = pd.DataFrame(columns = ['b'+str(i) for i in range(1,4)])
    for t in factors.loc[i+relativedelta(days=1):i+relativedelta(months=1,days=1),:].index:
        for j in forecast.columns:
            forecast.loc[t,j] = factors.shift().loc[t,j]
    return forecast

* forecast_b stores t+1m forecasted $\beta$s on the 20 dates using Random Walk Model.

In [35]:
forecast_b2 = {}
for i in days:
    forecast_b2[i] = forecast_RW(i, factors)

### Forecast yields with Random Walk $\beta$

In [36]:
forecast_y2 = {}
for i in days:
    forecast_y2[i] = DNS2Y(forecast_b2[i], lamb, month)

* forecast_y stores t+1m forecasted CMT yields on the 20 dates using Random Walk Model.

In [37]:
forecast_y2[days[0]] # Forecast t+1m CMTs Rates from 2013-04-04

,1 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
2013-04-05,0.191921,0.126414,0.0549061,-0.0100004,0.0673386,0.287319,0.830185,1.31228,1.8328,2.58666,2.85019
2013-04-08,0.176005,0.117097,0.0536124,-0.00079894,0.0823027,0.296722,0.817038,1.27615,1.7706,2.48569,2.73558
2013-04-09,0.17339,0.11536,0.0532373,0.00164325,0.0903789,0.310019,0.838875,1.30413,1.80456,2.52784,2.78056
2013-04-10,0.177024,0.117112,0.0527007,-0.00189315,0.0851733,0.306367,0.841589,1.31334,1.82116,2.55542,2.812
2013-04-11,0.179591,0.11974,0.0557526,0.00294494,0.0958526,0.324084,0.872824,1.35529,1.87413,2.62391,2.88588
2013-04-12,0.177264,0.117201,0.0529057,-0.000473577,0.0914493,0.318881,0.866454,1.34815,1.86628,2.61512,2.87677
2013-04-15,0.167853,0.10783,0.0431939,-0.012005,0.0735003,0.292996,0.82513,1.29451,1.79992,2.53083,2.78624
2013-04-16,0.164837,0.105928,0.0425331,-0.0114443,0.0731462,0.28939,0.813239,1.27517,1.77251,2.49169,2.743
2013-04-17,0.171884,0.113563,0.0510489,-0.00118633,0.0866851,0.305824,0.834229,1.29934,1.79975,2.52306,2.7758
2013-04-18,0.166352,0.109986,0.0498919,0.000968389,0.0912385,0.309575,0.832937,1.29255,1.78657,2.50029,2.74965


### Evaluation

In [38]:
def forecast_performance(forecast_set, data_set = CMTs):
    forecast_error = data_set.loc[forecast_set.index,:] - forecast_set
    forecast_perform = pd.DataFrame(columns = ['mean','std','RMSE'])
    forecast_perform.loc[:,'mean'] = forecast_error.mean()
    forecast_perform.loc[:,'std'] = forecast_error.std()
    forecast_perform.loc[:,'RMSE'] = np.sqrt((forecast_error**2).mean())
    
    return forecast_perform

#### Out-of-sample forecast performance of AR(1)

In [39]:
# t+5d forceast performance on 2013-04-04
forecast_performance(forecast_y[days[0]].head(5), CMTs)

,mean,std,RMSE
1 MO,-0.118173,0.012897,0.118735
3 MO,-0.048242,0.004546,0.048413
6 MO,0.048091,0.001806,0.048118
1 YR,0.128215,0.008788,0.128455
2 YR,0.153445,0.010053,0.153709
3 YR,0.034128,0.010511,0.035399
5 YR,-0.135244,0.020329,0.136461
7 YR,-0.156314,0.032240,0.158951
10 YR,-0.051563,0.048145,0.067179
20 YR,-0.002661,0.061049,0.054669


In [40]:
# t+10d forceast performance on 2013-04-04
forecast_performance(forecast_y[days[0]].head(10), CMTs)

,mean,std,RMSE
1 MO,-0.119536,0.011768,0.120056
3 MO,-0.049404,0.005167,0.049647
6 MO,0.045932,0.004071,0.046094
1 YR,0.127672,0.009965,0.128022
2 YR,0.149824,0.011818,0.150243
3 YR,0.030715,0.011772,0.032682
5 YR,-0.137824,0.019769,0.139094
7 YR,-0.168826,0.030965,0.171362
10 YR,-0.069652,0.042384,0.080425
20 YR,-0.021092,0.051163,0.052922


In [41]:
# t+1m forceast performance on 2013-04-04
forecast_performance(forecast_y[days[0]], CMTs)

,mean,std,RMSE
1 MO,-0.120221,0.010632,0.120667
3 MO,-0.047991,0.005414,0.048281
6 MO,0.046789,0.005155,0.047059
1 YR,0.128264,0.007697,0.128483
2 YR,0.145499,0.011683,0.145945
3 YR,0.031148,0.013915,0.033979
5 YR,-0.136383,0.024012,0.138381
7 YR,-0.170018,0.032611,0.172971
10 YR,-0.071749,0.040712,0.082015
20 YR,-0.016455,0.049419,0.050958


* From above, we can see the more time interval we forecast, generally the more RMSE it will be.

#### Out-of-sample forecast performance of Random Walk

In [42]:
# t+5d forceast performance on 2013-04-04
forecast_performance(forecast_y2[days[0]].head(5), CMTs)

,mean,std,RMSE
1 MO,-0.121586,0.013381,0.122174
3 MO,-0.051145,0.004476,0.051301
6 MO,0.045958,0.001255,0.045972
1 YR,0.127621,0.008780,0.127862
2 YR,0.155791,0.010757,0.156088
3 YR,0.039098,0.011163,0.040353
5 YR,-0.126102,0.020932,0.127485
7 YR,-0.144238,0.033278,0.147277
10 YR,-0.036650,0.050018,0.057834
20 YR,0.016096,0.064262,0.059689


In [43]:
# t+10d forceast performance on 2013-04-04
forecast_performance(forecast_y2[days[0]].head(10), CMTs)

,mean,std,RMSE
1 MO,-0.122612,0.011907,0.123131
3 MO,-0.052023,0.005070,0.052245
6 MO,0.044022,0.003854,0.044173
1 YR,0.127225,0.009987,0.127577
2 YR,0.152293,0.012233,0.152735
3 YR,0.035882,0.012265,0.037722
5 YR,-0.128250,0.020956,0.129782
7 YR,-0.156090,0.032241,0.159059
10 YR,-0.053828,0.044046,0.068143
20 YR,-0.001051,0.053893,0.051138


In [44]:
# t+1m forceast performance on 2013-04-04 model:Random Walk
forecast_performance(forecast_y2[days[0]], CMTs)

,mean,std,RMSE
1 MO,-0.122962,0.011024,0.123432
3 MO,-0.050321,0.005607,0.050618
6 MO,0.045125,0.005347,0.045426
1 YR,0.128054,0.007874,0.128285
2 YR,0.148391,0.011940,0.148848
3 YR,0.037052,0.014514,0.039667
5 YR,-0.125381,0.025215,0.127773
7 YR,-0.155270,0.033807,0.158736
10 YR,-0.053292,0.042155,0.067323
20 YR,0.007106,0.052053,0.051293


* Compare two forecast methods on 2013-04-04, it is easy to find that **the AR(1) model has more accuracy (lower RMSE) than Random Walk model**.

#### Compare two methods on different dates

In [45]:
from IPython.display import display_html
def display_side_by_side(*args):
    # Display DataFrames side by side, origin: https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [46]:
for i in range(20):
    df1 = forecast_performance(forecast_y[days[i]], CMTs)
    df2 = forecast_performance(forecast_y2[days[i]], CMTs)
    display_side_by_side(df1,df2)

,mean,std,RMSE
1 MO,-0.120221,0.010632,0.120667
3 MO,-0.047991,0.005414,0.048281
6 MO,0.046789,0.005155,0.047059
1 YR,0.128264,0.007697,0.128483
2 YR,0.145499,0.011683,0.145945
3 YR,0.031148,0.013915,0.033979
5 YR,-0.136383,0.024012,0.138381
7 YR,-0.170018,0.032611,0.172971
10 YR,-0.071749,0.040712,0.082015
20 YR,-0.016455,0.049419,0.050958


,mean,std,RMSE
1 MO,-0.114240,0.017464,0.115504
3 MO,-0.041120,0.007924,0.041841
6 MO,0.047345,0.011272,0.048606
1 YR,0.117271,0.014975,0.118179
2 YR,0.128630,0.025754,0.131063
3 YR,0.030855,0.026185,0.040063
5 YR,-0.085116,0.041313,0.094182
7 YR,-0.104266,0.050256,0.115225
10 YR,-0.013819,0.051869,0.052472
20 YR,0.027462,0.057632,0.062590


,mean,std,RMSE
1 MO,-0.119349,0.010550,0.119794
3 MO,-0.047846,0.005232,0.048118
6 MO,0.046649,0.005013,0.046905
1 YR,0.127681,0.007688,0.127901
2 YR,0.143025,0.011601,0.143473
3 YR,0.027616,0.013752,0.030711
5 YR,-0.142292,0.024897,0.144356
7 YR,-0.176420,0.032728,0.179294
10 YR,-0.078536,0.040235,0.087825
20 YR,-0.023884,0.048389,0.052967


,mean,std,RMSE
1 MO,-0.117042,0.016479,0.118142
3 MO,-0.042199,0.007148,0.042772
6 MO,0.048006,0.010840,0.049158
1 YR,0.119568,0.014281,0.120377
2 YR,0.132380,0.023073,0.134281
3 YR,0.033320,0.024763,0.041161
5 YR,-0.086719,0.042221,0.096010
7 YR,-0.107040,0.051757,0.118359
10 YR,-0.014808,0.052684,0.053504
20 YR,0.028806,0.056592,0.062289


,mean,std,RMSE
1 MO,-0.119776,0.012005,0.120348
3 MO,-0.050572,0.006454,0.050963
6 MO,0.045778,0.005203,0.046059
1 YR,0.126620,0.008537,0.126894
2 YR,0.141712,0.010750,0.142100
3 YR,0.023351,0.011988,0.026118
5 YR,-0.146240,0.020946,0.147662
7 YR,-0.177719,0.028646,0.179905
10 YR,-0.080515,0.033586,0.086931
20 YR,-0.026606,0.040891,0.047962


...

,mean,std,RMSE
1 MO,-0.122686,0.009646,0.123046
3 MO,-0.044193,0.005745,0.044547
6 MO,0.051630,0.005293,0.051888
1 YR,0.129238,0.007001,0.129418
2 YR,0.142980,0.015852,0.143814
3 YR,0.037006,0.017944,0.040940
5 YR,-0.114923,0.031324,0.118919
7 YR,-0.143620,0.039428,0.148685
10 YR,-0.045450,0.043896,0.062457
20 YR,0.014642,0.048139,0.049207


,mean,std,RMSE
1 MO,-0.115581,0.013995,0.116383
3 MO,-0.046435,0.010861,0.047626
6 MO,0.044736,0.005540,0.045060
1 YR,0.127748,0.010448,0.128154
2 YR,0.132417,0.013896,0.133108
3 YR,0.020244,0.012420,0.023587
5 YR,-0.123516,0.026284,0.126144
7 YR,-0.129121,0.039968,0.134869
10 YR,-0.030873,0.035066,0.046057
20 YR,0.013453,0.033600,0.035405


,mean,std,RMSE
1 MO,-0.122451,0.010581,0.122888
3 MO,-0.045261,0.005896,0.045627
6 MO,0.050894,0.005302,0.051158
1 YR,0.130394,0.007569,0.130604
2 YR,0.145544,0.015932,0.146376
3 YR,0.038152,0.016594,0.041460
5 YR,-0.116628,0.028074,0.119817
7 YR,-0.147888,0.035616,0.151935
10 YR,-0.050877,0.039794,0.064056
20 YR,0.007252,0.044059,0.043697


,mean,std,RMSE
1 MO,-0.122212,0.010821,0.122669
3 MO,-0.044718,0.006534,0.045172
6 MO,0.049650,0.007534,0.050194
1 YR,0.125434,0.010253,0.125835
2 YR,0.138402,0.017231,0.139424
3 YR,0.032129,0.019711,0.037469
5 YR,-0.108617,0.033980,0.113587
7 YR,-0.134983,0.043249,0.141455
10 YR,-0.038940,0.046621,0.059961
20 YR,0.013826,0.049844,0.050671


,mean,std,RMSE
1 MO,-0.090491,0.018533,0.092281
3 MO,-0.037779,0.007696,0.038518
6 MO,0.034054,0.012407,0.036142
1 YR,0.095662,0.019014,0.097446
2 YR,0.100806,0.030550,0.105122
3 YR,0.015970,0.028418,0.032002
5 YR,-0.056250,0.042464,0.069867
7 YR,-0.072999,0.047974,0.086722
10 YR,-0.002512,0.048529,0.047426
20 YR,0.027260,0.053648,0.059027


* From the comparsion tables above, **generally speaking, the AR(1) model has more out-of-sample accuracy than Random Walk model**. But the real accuracy **depends on specific dates and yield maturities**.
* **Solution to Step 3**: Generally speaking, these forecasts perform better than those with generated under a Random Walk.